In [ ]:
from dotenv import load_dotenv, find_dotenv
from langchain_community.graphs import Neo4jGraph

load_dotenv(find_dotenv())
db = Neo4jGraph()

In [ ]:
from langchain_openai import ChatOpenAI
from graph import calculate_risk_coefficients, car_analysis, go_to_searcher
from guardian import Guardian,  PictureAnalyzer
from query_generator import create_query_generator
from langgraph.prebuilt import ToolNode

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)
picture_analysis = PictureAnalyzer(llm).create_picture_analyzer()
query_gen = create_query_generator(llm, db)

In [ ]:
guardian = Guardian(llm, [go_to_searcher])
llm_with_guardian = guardian.create_guardian()
search_node = ToolNode([go_to_searcher])

In [ ]:
tool_node = ToolNode([calculate_risk_coefficients, car_analysis])

In [ ]:
from caller import Caller

mini_llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

caller = Caller(mini_llm, [calculate_risk_coefficients, car_analysis]).create_caller()

app_tools = {
    'db': db,
    'caller': caller,
    'guardian': llm_with_guardian,
    'cypher': query_gen,
    'picture': picture_analysis,
}

In [ ]:
from graph import State, compile_graph
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph

graph = StateGraph(State)
memory = MemorySaver()

app = compile_graph(graph, memory, search_node, tool_node)

In [ ]:
from IPython.display import Image, display

try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

config = {"configurable": {"thread_id": "1"}, "recursion_limit": 15}
config['configurable'].update(app_tools)

In [ ]:
user_input = "Dimmi di Vito Teci"

events = app.astream(
    {"messages": ("user", user_input), "is_picture": False}, config=config, stream_mode="values",
)
async for event in events:
    event["messages"][-1].pretty_print()
    final_message = event

In [ ]:
image_url = "https://immagini.alvolante.it/sites/default/files/styles/anteprima_640/public/dasapere_galleria/2024/07/targa-italiana-987.jpg"
import httpx
import base64
image_data = base64.b64encode(httpx.get(image_url).content).decode("utf-8")
image = f"{image_data}"

In [ ]:
events = app.astream(
    {"messages": ("user", image), "is_picture": True}, config=config, stream_mode="values",
)
async for event in events:
    event["messages"][-1].pretty_print()
    final_message = event

In [ ]:
print(final_message["messages"][-1].content)

In [ ]:
snapshot = app.get_state(config).values
from pprint import pprint
pprint(snapshot)

In [ ]:
app.get_state(config).values['messages'].extend(final_message['messages'][-1])

In [ ]:
pprint(app.get_state(config).values)